In [18]:
import pandas as pd

In [108]:
df = [pd.DataFrame(n_grams[j], columns=['col_%d' % i for i in range(len(n_grams[j][0]))]) for j in range(len(n_grams))]

In [114]:
def getProb(df, index):
    df_group = df.groupby(['col_%d' % i for i in range(index)])[['col_%d' % i for i in range(index)]].count().rename(\
                        columns={'col_%d' % i:'count_%d' % i for i in range(index)}).reset_index()
    
    df_group = df_group[['col_%d' % i for i in range(index)] + ['count_0']]
    
    df_group['prob'] = np.log(df_group['count_0']/df_group['count_0'].sum())
    
    return df_group

In [117]:
df_group = getProb(df[3], 3)

In [120]:
df_group.values[0]

array([  3.00000000e+00,   5.25000000e+02,   1.49260000e+04,
         1.00000000e+00,  -1.21021555e+01])

In [95]:
#df[2].groupby(['col_0', 'col_1', 'col_2'])[['col_0', 'col_1', 'col_2']].count().rename(\
#                                        columns={'col_0':'count_1', 'col_1':'count_1', 'col_2':'count_2'}).reset_index()

In [6]:
import tensorflow as tf
import numpy as np

import pickle

tf.reset_default_graph()

vocab_size = 17322 + 1
beam_width = 5

time = 0
n_grams = pickle.load(open('../n_grams_test.pkl', 'rb'))

n_grams_tf = [tf.constant(n_grams[i]) for i in range(2)]

In [7]:
time = tf.convert_to_tensor(time, name="time")

In [12]:
def cond(time, n_grams):
    return time<2

In [13]:
def body(time, n_grams):
    n_grams = n_grams_tf[time]
    
    return time+1, n_grams

In [14]:
increment_test = tf.while_loop(cond, body, loop_vars=[time, n_grams_tf[0]],
                        parallel_iterations=1)

TypeError: list indices must be integers or slices, not Tensor

In [16]:
with tf.Session() as sess:
    q = sess.run(increment_test)

ValueError: Fetch argument <tf.Tensor 'while/Exit:0' shape=() dtype=int32> cannot be interpreted as a Tensor. (Tensor Tensor("while/Exit:0", shape=(), dtype=int32) is not an element of this graph.)

In [12]:
q

10

In [2]:
#import pandas s pd
#df = pd.DataFrame(n_grams[time], columns=['a', 'b'])

In [3]:
#beam = tf.constant([[6178, 5729, 15173], 
#                    [2341, 16073, 5683], 
#                    [434, 9055, 2692], 
#                    [6178, 14056, 7441]]) #Current predicted steps, will specify at run time

In [23]:
beam = tf.constant([[]], dtype=tf.int32) #Current predicted steps, will specify at run time

In [24]:
beam_pad = tf.pad(beam, [[0, 0], [0, 1]], mode='CONSTANT', constant_values=0)

initial_beam_step = tf.constant(0)

n_beams = tf.constant(beam_width) #will specify at run time

#When you specify the axis of concat such a rank must exist! 
#Thus cannot specify axis=1 if the rank is 0!
scatter_base = tf.constant([vocab_size]) #Size of dict for scatter base will specify at run time
concat_base = tf.constant([[1.0 for i in range(vocab_size)]], dtype=tf.float64) #concat base will specify vocab size at run time

In [37]:
def anti_lm_body(test, n_grams_tf, beam_pad, current_beam, n_beams):
    #This loops on each beam individually
      
    def grab_probs(n_grams_tf, y_equal_2): 
    
        #print ('Grab Probabilities')
        y_args = tf.where(y_equal_2)

        #Grab the n_gram sequences that are matched
        y_diff_gather = tf.gather(n_grams_tf, y_args)
        #Take the last token in each sequence and count their unique occurrences

        last_token_pos = tf.shape(y_diff_gather)[-1]

        y_last_token = y_diff_gather[:,0][:,last_token_pos-1]
        y_last_token_unique = tf.unique_with_counts(y_last_token)

        total_count = tf.reduce_sum(y_last_token_unique.count)

        indices = tf.reshape(y_last_token_unique.y, [tf.shape(y_last_token_unique.y)[0], 1])

        test_add_result = tf.scatter_nd(indices=indices, 
                    updates=y_last_token_unique.count, shape=scatter_base)/total_count

        test_add_result = tf.log(test_add_result + 10e-10)

        return tf.reshape(test_add_result, [1, vocab_size])

    def dump_zeros(n_grams_tf, y_equal_2): 
        #print ('No matches found')
        return tf.constant([[0. for i in range(vocab_size)]], dtype=tf.float64)

    #Find where current beam matches n_gram sequence up to current seq pos, cast as int
    y_test = tf.to_int32(tf.equal(n_grams_tf, beam_pad[current_beam]))
    #Reduce across the length of the beam 
    y_test_reduce_sum = tf.reduce_sum(y_test, axis=1)

    y_empty = tf.reduce_sum(y_test_reduce_sum)

    #Find args where the beam is matched to the n_gram combinations
    y_equal_2  = tf.equal(time, y_test_reduce_sum)

    #Why does cond proceed down both paths?
    test_add = tf.cond(tf.equal(0, tf.reduce_sum(tf.to_int32(y_equal_2))),
        true_fn=lambda: dump_zeros(n_grams_tf, y_equal_2),
        false_fn=lambda : grab_probs(n_grams_tf, y_equal_2))   
   
    test = tf.concat([test, tf.reshape(test_add, [1, vocab_size])], axis=0)
    #test = test_add
    return test, n_grams_tf, beam_pad, current_beam+1, n_beams

def anti_lm_condition(test, n_grams_tf, beam_pad, current_beam, n_beams):
    return current_beam < n_beams #number of beams, i.e. beam width, will specify at run time

anti_lm_outputs = tf.while_loop(anti_lm_condition, anti_lm_body, 
                        [concat_base, n_grams_tf, beam_pad, initial_beam_step, n_beams], 
                        shape_invariants=[tf.TensorShape([None, vocab_size]), 
                                          n_grams_tf.get_shape(),
                                      beam_pad.get_shape(), initial_beam_step.get_shape(), n_beams.get_shape()])

#anti_beam_probs = anti_lm_outputs[0]

In [38]:
with tf.Session() as sess:
    tf.initialize_all_variables().run()
    #q = sess.run(outputs[0].stack())
    q = sess.run(anti_lm_outputs)

Instructions for updating:
Use `tf.global_variables_initializer` instead.


InvalidArgumentError: slice index 1 of dimension 0 out of bounds.
	 [[Node: while_7/strided_slice = StridedSlice[Index=DT_INT32, T=DT_INT32, begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=1, _device="/job:localhost/replica:0/task:0/gpu:0"](while_7/Identity_2, while_7/strided_slice/stack, while_7/strided_slice/stack_1, while_7/strided_slice/stack_2)]]
	 [[Node: while_7/cond/strided_slice_2/_29 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_130_while_7/cond/strided_slice_2", tensor_type=DT_INT32, _device="/job:localhost/replica:0/task:0/cpu:0"](^_cloopwhile_7/cond/Gather/_1)]]

Caused by op 'while_7/strided_slice', defined at:
  File "c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\runpy.py", line 85

In [39]:
debug

> c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\client\session.py(1340)_do_call()
   1338         except KeyError:
   1339           pass
-> 1340       raise type(e)(node_def, op, message)
   1341 
   1342   def _extend_graph(self):

ipdb> up
> c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\client\session.py(1321)_do_run()
   1319     if handle is None:
   1320       return self._do_call(_run_fn, self._session, feeds, fetches, targets,
-> 1321                            options, run_metadata)
   1322     else:
   1323       return self._do_call(_prun_fn, self._session, handle, feeds, fetches)

ipdb> up
> c:\users\euix\anaconda3\envs\tensorflow-gpu\lib\site-packages\tensorflow\python\client\session.py(1124)_run()
   1122     if final_fetches or final_targets or (handle and feed_dict_tensor):
   1123       results = self._do_run(handle, final_targets, final_fetches,
-> 1124                              feed_dict_tensor, o

In [17]:
sorted(q[0][0], reverse=True)

[-3.5057981520005326,
 -3.6137900279306905,
 -3.7773508321303284,
 -3.8385816955223757,
 -3.9774291008340366,
 -4.1923154411539851,
 -4.326896252403424,
 -4.3772836556546935,
 -4.3777254493090449,
 -4.4073240201650101,
 -4.4303452927600429,
 -4.4903296858679767,
 -4.4952875537856674,
 -4.5933849051365589,
 -4.7269162836427405,
 -4.7542282353764991,
 -4.8037269511269498,
 -5.070430756593181,
 -5.1314419271946861,
 -5.1513572334909519,
 -5.1934172152011087,
 -5.2478174821933825,
 -5.2531057008067235,
 -5.2594886999776884,
 -5.2691402475641258,
 -5.2909275974704233,
 -5.3120747363148171,
 -5.3711538590061911,
 -5.3880014259240054,
 -5.3940878706448414,
 -5.4339436971310269,
 -5.4441608973254274,
 -5.459685138208533,
 -5.4820987277182356,
 -5.491475887266426,
 -5.5496640304134601,
 -5.590426577104612,
 -5.5948941931650005,
 -5.6903536108910187,
 -5.7254449201309221,
 -5.7271470476791873,
 -5.7600504385136926,
 -5.7618125529473661,
 -5.7724509477521648,
 -5.7886238017587823,
 -5.80322259637

In [16]:
np.max(q[0]), np.max(q[1]), np.max(q[2]), np.max(q[3])

(1.0, 17322, 1167832, 5)

In [17]:
np.argsort(q[0])

array([[    0, 11538, 11539, ...,  5783,  5769, 17321],
       [    0, 11494, 11495, ..., 14250, 15173,  3380],
       [    0, 11538, 11539, ...,  5783,  5769, 17321],
       [    0, 11538, 11539, ...,  5783,  5769, 17321],
       [    0, 11538, 11539, ...,  5783,  5769, 17321],
       [    0, 11537, 11538, ...,  5775, 17321, 13459]], dtype=int64)